# Analysis

## Intro

## Crime Rate (by State) - Heat Map

## Income (by State) - Heat Map

In [7]:
import os
import pandas as pd

import plotly.plotly as py

In [2]:
median_income_df = pd.read_csv(os.path.join('formatted','median_income.csv'))
median_income_df.head()

,State,2016,2015,2014,2013,2012,2011,2010,2009,2008,...,1993,1992,1991,1990,1989,1988,1987,1986,1985,1984
0,United States,59039,56516,53657,53585,51017,50054,49276,49777,50303,...,31241,30636,30126,29943,28906,27225,26061,24897,23618,22415
1,AL,47221,44509,42278,47320,43464,42590,40933,39980,44476,...,25082,25808,24346,23357,21284,19948,19734,19132,18333,17310
2,AK,75723,75112,67629,72472,63648,57431,57848,61604,63989,...,42931,41802,40612,39298,36006,33103,33233,31356,34782,32356
3,AZ,57100,52248,49254,52611,47044,48621,46896,45739,46914,...,30510,29358,30737,29224,28552,26435,26749,25500,23877,21425
4,AR,45907,42798,44922,39376,39018,41302,38587,36538,39586,...,23039,23882,23435,22786,21433,20172,18827,18730,17451,15674


In [ ]:
# Create initial heat map




## Unemployment (by State) - Map